# Navigation

---

In this notebook, we use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  

In [ ]:
from unityagents import UnityEnvironment
import torch
import numpy as np
import matplotlib.pyplot as plt

import rl_lib
from rl_lib.replay import ReplayBuffer
from rl_lib.agent import Agent, DQNExtensions
from rl_lib.training import dqn

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [ ]:
env = UnityEnvironment(file_name="Banana.app")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [ ]:
# get information about the environment
action_size = brain.vector_action_space_size

env_info = env.reset(train_mode=True)[brain_name]
state_size = len(env_info.vector_observations[0])

### 2. Train the agent

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to train the agent to navigate through the environment. The agent tries to collect many yellow bananas while avoiding blue bananas.

In [ ]:
# set-up replay buffer


ReplayBufferInput = {
    "action_size": action_size,  # dimension of each action
    "buffer_size": int(1e5),  # replay buffer size
    "batch_size": 64,  # minibatch size
    "random_seed": 0,
    "alpha": 0.7,
    "beta": 0.5
}

replay_buffer = ReplayBuffer(**ReplayBufferInput)

In [ ]:
# set-up the agent 

AgentInput = {
    "state_size": state_size, # dimension of each state
    "action_size": action_size, # dimension of each action
    "seed": 0, # random seed
    "replay_buffer": replay_buffer, # replay buffer
    "gamma": 0.99, # discount factor
    "tau": 1e-3, # for soft update of target parameters
    "learning_rate": 5e-4, # learning rate
    "update_local_every": 2, # how often to update the local network
    "update_target_every": 4, # how often to update the target network
    "dqn_extensions": [DQNExtensions.DoubleDQN] # if empty, the standard DQN algorithm is used for learning.
    
}

agent = Agent(**AgentInput)

In [ ]:
# start training
scores = dqn(env, brain_name, agent, min_avg_score=10., eps_end=0.01, eps_decay=0.995, continue_learning=False)

### 3. Visualize the results

In [ ]:
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

In [ ]:
# see the agent acting in the environment

agent.qnetwork_local.load_state_dict(torch.load('checkpoint.pth'))

env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = agent.act(state)                      # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

When finished, you can close the environment.

In [ ]:
env.close()